In [ ]:
"""
001_prediction_function.py

This file provides two main functions:
    1. train_model(sentiment_data, return_data)
    2. predict_returns(model_info, sentiment_data_today, stock_universe_today, historical_data=None)

The code processes Reddit sentiment data, creates aggregated features, and trains a PyTorch feedforward neural network (with GPU support when available)
to predict next-day stock returns.
"""

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from datetime import datetime, time, timedelta
from sklearn.model_selection import TimeSeriesSplit
import os

# Set device for GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

####################################
# Helper Functions
####################################

def convert_return(x):
    """
    Converts a return value to a float.
    If x is a string ending in '%', removes the '%' and divides by 100.
    """
    if isinstance(x, str):
        x = x.strip()
        if x.endswith('%'):
            return float(x[:-1].strip()) / 100.0
        else:
            return float(x)
    else:
        return float(x)

def preprocess_sentiment_data(sentiment_data):
    """
    Preprocess sentiment data:
      - Convert 'Received_Time' to a timezone-aware datetime (UTC) then to US/Eastern.
      - Create a 'Date' column by shifting posts received after 4:00 PM (EST) to the next day.
      - Ensure 'Ticker' is uppercase.
    
    Parameters
    ----------
    sentiment_data : DataFrame
        Raw sentiment data containing at least 'Received_Time' and 'Ticker'.
    
    Returns
    -------
    df : DataFrame
        Processed sentiment data with added 'Received_Time_EST' and 'Date' columns.
    """
    df = sentiment_data.copy()
    if 'Received_Time' not in df.columns:
        raise ValueError("Column 'Received_Time' not found in the sentiment data.")
    
    df['Received_Time'] = pd.to_datetime(df['Received_Time'], utc=True)
    df['Received_Time_EST'] = df['Received_Time'].dt.tz_convert('America/New_York')
    df['local_date'] = df['Received_Time_EST'].dt.date
    cutoff = time(16, 0)  # 4:00 PM cutoff
    df['Date'] = df['Received_Time_EST'].apply(
        lambda x: pd.to_datetime(x.date() + timedelta(days=1)) if x.time() > cutoff else pd.to_datetime(x.date())
    )
    if 'Ticker' in df.columns:
        df['Ticker'] = df['Ticker'].str.upper()
    return df

def create_features(df):
    """
    Create advanced daily features by aggregating sentiment data for each Ticker and Date.
    
    Features include:
      - Basic statistics: mean, std, count
      - Advanced sentiment metrics: weighted sentiment, sentiment skewness
      - Volume indicators: log_post_count, normalized_post_volume
      - Temporal patterns: am_pm_sentiment_ratio, intraday_sentiment_volatility
      - Probability-based measures: sentiment_confidence_product, pos_neg_ratio
      - Source and topic relevance: relevance_weighted_sentiment
      - Author-based metrics: unique_author_count, avg_author_sentiment
    
    Parameters
    ----------
    df : DataFrame
        Preprocessed sentiment data with columns including 
        ['Ticker', 'Date', 'Sentiment', 'Confidence', 'Prob_POS', 'Prob_NTR', 'Prob_NEG',
         'Relevance', 'SourceWeight', 'TopicWeight', 'Author'].
    
    Returns
    -------
    features : DataFrame
        Aggregated features for each Ticker and Date.
    """
    # If the dataframe is empty, return an empty dataframe with all required columns
    if df.empty:
        empty_df = pd.DataFrame(columns=['Ticker', 'Date', 'sentiment_mean', 'sentiment_std', 'post_count', 
                                         'sentiment_skew', 'sentiment_median', 'avg_confidence', 'confidence_std',
                                         'avg_prob_pos', 'max_prob_pos', 'avg_prob_ntr', 'avg_prob_neg', 
                                         'max_prob_neg', 'avg_source_weight', 'avg_topic_weight', 'avg_relevance',
                                         'total_relevance', 'unique_author_count', 'morning_post_ratio',
                                         'market_hours_ratio', 'log_post_count', 'pos_neg_ratio',
                                         'sentiment_confidence_product', 'weighted_sentiment',
                                         'author_avg_sentiment', 'am_pm_sentiment_ratio',
                                         'intraday_sentiment_volatility'])
        return empty_df
    
    required_columns = ['Ticker', 'Date', 'Sentiment', 'Confidence', 'Prob_POS', 
                        'Prob_NTR', 'Prob_NEG', 'Relevance', 'SourceWeight', 'TopicWeight']
    for col in required_columns:
        if col not in df.columns:
            if col in ['Ticker', 'Date', 'Sentiment']:
                raise ValueError(f"Required column {col} is missing in the data.")
            else:
                df[col] = 0
    
    # Add Author column if missing
    if 'Author' not in df.columns:
        df['Author'] = 'unknown'
    
    # Create time-of-day features
    df['hour'] = df['Received_Time_EST'].dt.hour
    df['is_am'] = df['hour'] < 12
    df['is_market_hours'] = (df['hour'] >= 9.5) & (df['hour'] <= 16)
    
    # Basic aggregations
    agg_funcs = {
        'Sentiment': ['mean', 'std', 'count', 'skew', 'median'],
        'Confidence': ['mean', 'std'],
        'Prob_POS': ['mean', 'max'],
        'Prob_NTR': ['mean'],
        'Prob_NEG': ['mean', 'max'],
        'Relevance': ['mean', 'sum'],
        'SourceWeight': ['mean'],
        'TopicWeight': ['mean'],
        'Author': ['nunique'],
        'is_am': ['mean'],
        'is_market_hours': ['mean']
    }
    
    grouped = df.groupby(['Ticker', 'Date']).agg(agg_funcs)
    grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]
    grouped = grouped.reset_index()
    
    # Rename columns for clarity
    rename_dict = {
        'Sentiment_mean': 'sentiment_mean',
        'Sentiment_std': 'sentiment_std',
        'Sentiment_count': 'post_count',
        'Sentiment_skew': 'sentiment_skew',
        'Sentiment_median': 'sentiment_median',
        'Confidence_mean': 'avg_confidence',
        'Confidence_std': 'confidence_std',
        'Prob_POS_mean': 'avg_prob_pos',
        'Prob_POS_max': 'max_prob_pos',
        'Prob_NTR_mean': 'avg_prob_ntr',
        'Prob_NEG_mean': 'avg_prob_neg',
        'Prob_NEG_max': 'max_prob_neg',
        'SourceWeight_mean': 'avg_source_weight',
        'TopicWeight_mean': 'avg_topic_weight',
        'Relevance_mean': 'avg_relevance',
        'Relevance_sum': 'total_relevance',
        'Author_nunique': 'unique_author_count',
        'is_am_mean': 'morning_post_ratio',
        'is_market_hours_mean': 'market_hours_ratio'
    }
    grouped = grouped.rename(columns=rename_dict)
    
    # Calculate additional derived features
    
    # Volume indicators
    grouped['log_post_count'] = np.log1p(grouped['post_count'])
    
    # Advanced sentiment metrics
    grouped['pos_neg_ratio'] = grouped['avg_prob_pos'] / (grouped['avg_prob_neg'] + 0.001)
    grouped['sentiment_confidence_product'] = grouped['sentiment_mean'] * grouped['avg_confidence']
    
    # Calculate weighted sentiment metrics
    def weighted_sentiment_func(sub_df):
        if sub_df['Sentiment'].count() == 0:
            return 0
        return (sub_df['Sentiment'] * sub_df['Relevance']).sum() / sub_df['Relevance'].sum() if sub_df['Relevance'].sum() > 0 else 0
    
    def author_sentiment_avg(sub_df):
        return sub_df.groupby('Author')['Sentiment'].mean().mean() if sub_df['Author'].nunique() > 0 else 0
        
    # Time-based metrics
    def am_pm_sentiment_ratio(sub_df):
        am_sentiment = sub_df[sub_df['is_am']]['Sentiment'].mean() if not sub_df[sub_df['is_am']].empty else 0
        pm_sentiment = sub_df[~sub_df['is_am']]['Sentiment'].mean() if not sub_df[~sub_df['is_am']].empty else 0
        return am_sentiment / (pm_sentiment + 0.001) if pm_sentiment != 0 else 1
    
    def intraday_volatility(sub_df):
        return sub_df.groupby(['hour'])['Sentiment'].std().mean() if not sub_df.empty else 0
    
    # Special aggregations
    special_aggs = df.groupby(['Ticker', 'Date']).apply(
        lambda x: pd.Series({
            'weighted_sentiment': weighted_sentiment_func(x),
            'author_avg_sentiment': author_sentiment_avg(x),
            'am_pm_sentiment_ratio': am_pm_sentiment_ratio(x),
            'intraday_sentiment_volatility': intraday_volatility(x)
        })
    ).reset_index()
    
    # Merge special aggregations
    grouped = pd.merge(grouped, special_aggs, on=['Ticker', 'Date'], how='left')
    
    # Handle missing values
    grouped = grouped.fillna(0)
    
    return grouped

####################################
# PyTorch Neural Network Model
####################################

class FeedforwardNet(nn.Module):
    def __init__(self, input_dim, hidden_dims=[128, 64], dropout_rate=0.3):
        """
        A feedforward neural network with configurable hidden layers and dropout.
        
        Parameters
        ----------
        input_dim : int
            Number of input features
        hidden_dims : list
            List of hidden layer dimensions
        dropout_rate : float
            Dropout probability for regularization
        """
        super(FeedforwardNet, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = dim
        
        layers.append(nn.Linear(prev_dim, 1))
        
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

####################################
# Main Functions
####################################

def train_model(sentiment_data, return_data):
    """
    Train a model using sentiment features to predict next-day returns.
    
    Parameters
    ----------
    sentiment_data : DataFrame
        The Reddit sentiment data for training.
    return_data : DataFrame
        The stock return data for training.
    
    Returns
    -------
    model_info : dict
        Contains the trained PyTorch model, scaler, feature columns, and device information.
    """
    print("Preprocessing sentiment data...")
    sentiment_data = preprocess_sentiment_data(sentiment_data)
    print("Creating features from sentiment data...")
    features_df = create_features(sentiment_data)
    
    # Preprocess return_data: ensure Date is datetime, Ticker uppercase, and convert Return.
    return_data = return_data.copy()
    return_data['Date'] = pd.to_datetime(return_data['Date'])
    return_data['Ticker'] = return_data['Ticker'].str.upper()
    return_data['Return'] = return_data['Return'].apply(convert_return)
    
    print("Merging sentiment features with stock returns...")
    model_data = pd.merge(features_df, return_data[['Date', 'Ticker', 'Return']], on=['Date', 'Ticker'], how='inner')
    model_data = model_data.dropna(subset=['Return'])
    
    # Get all feature columns (excluding Date, Ticker, Return)
    all_feature_columns = [col for col in model_data.columns if col not in ['Date', 'Ticker', 'Return']]
    model_data[all_feature_columns] = model_data[all_feature_columns].fillna(0)
    
    # Sort data chronologically
    model_data = model_data.sort_values('Date')
    
    # Use time series cross-validation for proper evaluation
    unique_dates = np.sort(model_data['Date'].unique())
    n_splits = 5
    tscv = TimeSeriesSplit(n_splits=n_splits)
    date_indices = np.arange(len(unique_dates))
    
    best_val_loss = float('inf')
    best_model = None
    best_scaler = None
    
    for train_idx, val_idx in tscv.split(date_indices):
        train_dates = unique_dates[train_idx]
        val_dates = unique_dates[val_idx]
        
        train_data = model_data[model_data['Date'].isin(train_dates)]
        val_data = model_data[model_data['Date'].isin(val_dates)]
        
        X_train = train_data[all_feature_columns].values
        y_train = train_data['Return'].values.reshape(-1, 1)
        X_val = val_data[all_feature_columns].values
        y_val = val_data['Return'].values.reshape(-1, 1)
        
        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        
        X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
        X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
        y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)
        
        input_dim = X_train_tensor.shape[1]
        
        # Create model with improved architecture
        model_net = FeedforwardNet(
            input_dim=input_dim, 
            hidden_dims=[128, 64], 
            dropout_rate=0.3
        ).to(device)
        
        # Define loss and optimizer with learning rate decay
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model_net.parameters(), lr=0.001, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
        
        # Early stopping setup
        patience = 30
        best_fold_loss = float('inf')
        patience_counter = 0
        best_fold_model_state = None
        
        # Training loop
        print(f"Training fold {len(train_dates)} days -> {len(val_dates)} days...")
        epochs = 300
        for epoch in range(epochs):
            model_net.train()
            optimizer.zero_grad()
            outputs = model_net(X_train_tensor)
            loss = criterion(outputs, y_train_tensor)
            loss.backward()
            optimizer.step()
            
            # Validation
            model_net.eval()
            with torch.no_grad():
                val_outputs = model_net(X_val_tensor)
                val_loss = criterion(val_outputs, y_val_tensor)
                scheduler.step(val_loss)
            
            # Early stopping check
            if val_loss.item() < best_fold_loss:
                best_fold_loss = val_loss.item()
                patience_counter = 0
                best_fold_model_state = model_net.state_dict().copy()
            else:
                patience_counter += 1
            
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
                
            if epoch % 20 == 0:
                print(f"Epoch {epoch:3d} | Train Loss: {loss.item():.6f} | Val Loss: {val_loss.item():.6f}")
        
        # Restore best model from this fold
        model_net.load_state_dict(best_fold_model_state)
        
        # Final validation evaluation
        model_net.eval()
        with torch.no_grad():
            final_val_outputs = model_net(X_val_tensor)
            final_val_loss = criterion(final_val_outputs, y_val_tensor)
        
        print(f"Fold final validation loss: {final_val_loss.item():.6f}")
        
        # Check if this is the best model across folds
        if final_val_loss.item() < best_val_loss:
            best_val_loss = final_val_loss.item()
            best_model = model_net
            best_scaler = scaler
    
    # Now train a final model on the complete dataset
    print("Training final model on complete dataset...")
    X_all = model_data[all_feature_columns].values
    y_all = model_data['Return'].values.reshape(-1, 1)
    
    # Scale all data with the best scaler from cross-validation
    X_all_scaled = best_scaler.transform(X_all)
    X_all_tensor = torch.tensor(X_all_scaled, dtype=torch.float32).to(device)
    y_all_tensor = torch.tensor(y_all, dtype=torch.float32).to(device)
    
    # Initialize the final model with the same architecture as best_model
    final_model = FeedforwardNet(
        input_dim=X_all_tensor.shape[1],
        hidden_dims=[128, 64],
        dropout_rate=0.3
    ).to(device)
    
    # Copy weights from best model as starting point
    final_model.load_state_dict(best_model.state_dict())
    
    # Define loss and optimizer for final training
    final_optimizer = optim.Adam(final_model.parameters(), lr=0.0005, weight_decay=1e-4)
    final_criterion = nn.MSELoss()
    
    # Final training loop - fewer epochs to avoid overfitting
    final_epochs = 100
    for epoch in range(final_epochs):
        final_model.train()
        final_optimizer.zero_grad()
        outputs = final_model(X_all_tensor)
        loss = final_criterion(outputs, y_all_tensor)
        loss.backward()
        final_optimizer.step()
        
        if epoch % 20 == 0:
            print(f"Final model - Epoch {epoch:3d} | Loss: {loss.item():.6f}")
    
    # Save feature importance (approximated by analyzing weights)
    with torch.no_grad():
        # Get weights from first layer
        first_layer_weights = final_model.net[0].weight.cpu().numpy()
        
        # Calculate importance by taking the absolute value of weights
        feature_importance = np.abs(first_layer_weights).mean(axis=0)
        
        # Create feature importance dictionary
        importance_dict = dict(zip(all_feature_columns, feature_importance))
        
        # Sort features by importance
        sorted_importance = dict(sorted(importance_dict.items(), key=lambda x: x[1], reverse=True))
        
        # Print top 10 features
        print("\nTop 10 most important features:")
        for i, (feat, imp) in enumerate(list(sorted_importance.items())[:10]):
            print(f"{i+1}. {feat}: {imp:.6f}")
    
    model_info = {
        'model': final_model,
        'scaler': best_scaler,
        'feature_columns': all_feature_columns,
        'device': device,
        'feature_importance': sorted_importance
    }
    
    print("Training complete. Model is ready.")
    return model_info

def predict_returns(model_info, sentiment_data_today, stock_universe_today, historical_data=None):
    """
    Generate predictions of next-day returns for all stocks in the universe.
    
    Parameters
    ----------
    model_info : dict
        Contains the trained model, scaler, and feature columns.
    sentiment_data_today : DataFrame
        New sentiment data (for a single day).
    stock_universe_today : list
        List of stock tickers available today.
    historical_data : dict, optional
        Not used in this implementation.
    
    Returns
    -------
    predictions : DataFrame
        A DataFrame with columns ['Ticker', 'Predicted_Return', 'Signal_Rank'].
    """
    # Handle empty inputs
    if sentiment_data_today.empty:
        # Return default predictions for the stock universe
        predictions = pd.DataFrame({
            'Ticker': [ticker.upper() for ticker in stock_universe_today],
            'Predicted_Return': [0.0] * len(stock_universe_today)
        })
        predictions['Signal_Rank'] = 0.5  # Neutral ranking for all stocks
        return predictions
    
    # Preprocess the sentiment data
    sentiment_data_today = preprocess_sentiment_data(sentiment_data_today)
    
    # Define current_date using the max date available in the preprocessed data
    if sentiment_data_today.empty:
        current_date = pd.Timestamp.today().normalize()
    else:
        current_date = sentiment_data_today['Date'].max()
    
    # Filter to today's sentiment data
    sentiment_today = sentiment_data_today[sentiment_data_today['Date'] == current_date].copy()
    
    # Create features from sentiment data
    features_today = create_features(sentiment_today) 
    
    # Ensure all tickers in the universe are included
    universe_upper = [t.upper() for t in stock_universe_today]
    
    # If features_today is empty or missing the 'Ticker' column, build a default features DataFrame
    if features_today.empty or 'Ticker' not in features_today.columns:
        default_data = []
        for t in universe_upper:
            default_row = {col: 0 for col in model_info['feature_columns']}
            default_row['Ticker'] = t
            default_row['Date'] = current_date
            default_data.append(default_row)
        features_today = pd.DataFrame(default_data)
    else:
        # Ensure the Ticker column is uppercase
        features_today['Ticker'] = features_today['Ticker'].str.upper()
        
        # Filter features to include only tickers in stock_universe_today
        features_today = features_today[features_today['Ticker'].isin(universe_upper)]
        
        # Add default rows for any tickers that are missing
        existing_tickers = set(features_today['Ticker'])
        missing_tickers = set(universe_upper) - existing_tickers
        
        if missing_tickers:
            default_data = []
            for t in missing_tickers:
                default_row = {col: 0 for col in model_info['feature_columns']}
                default_row['Ticker'] = t
                default_row['Date'] = current_date
                default_data.append(default_row)
            
            if default_data:
                default_df = pd.DataFrame(default_data)
                features_today = pd.concat([features_today, default_df], ignore_index=True)
    
    # Final safeguard: if features_today is still empty, force defaults for every ticker
    if features_today.empty:
        default_data = []
        for t in universe_upper:
            default_row = {col: 0 for col in model_info['feature_columns']}
            default_row['Ticker'] = t
            default_row['Date'] = current_date
            default_data.append(default_row)
        features_today = pd.DataFrame(default_data)
    
    # Make sure all required feature columns exist
    for col in model_info['feature_columns']:
        if col not in features_today.columns:
            features_today[col] = 0
    
    # Sort values safely (only if Ticker exists)
    if 'Ticker' in features_today.columns:
        features_today = features_today.sort_values('Ticker').reset_index(drop=True)
    
    # Prepare features for prediction
    X_today = features_today[model_info['feature_columns']].fillna(0).values
    
    if X_today.shape[0] == 0:
        # If no data is present, create a default matrix
        X_today = np.zeros((len(universe_upper), len(model_info['feature_columns'])))
        features_today = pd.DataFrame({
            'Ticker': universe_upper,
        })
        for col in model_info['feature_columns']:
            features_today[col] = 0

    # Scale features using the saved scaler
    X_today_scaled = model_info['scaler'].transform(X_today)
    X_today_tensor = torch.tensor(X_today_scaled, dtype=torch.float32).to(model_info['device'])
    
    # Generate predictions
    model = model_info['model']
    model.eval()
    with torch.no_grad():
        predictions_tensor = model(X_today_tensor)
    predictions_array = predictions_tensor.cpu().numpy().flatten()
    
    # Add a small random noise to break any ties
    predictions_array += np.random.normal(0, 1e-6, size=predictions_array.shape)
    
    # Create prediction DataFrame
    predictions = pd.DataFrame({
        'Ticker': features_today['Ticker'],
        'Predicted_Return': predictions_array
    })
    
    # Calculate percentile rank
    predictions['Signal_Rank'] = predictions['Predicted_Return'].rank(pct=True)
    
    # Ensure all columns are included and in the right order
    predictions = predictions[['Ticker', 'Predicted_Return', 'Signal_Rank']]
    
    return predictions

####################################
# Test Section (Runs if the script is executed directly)
####################################
if __name__ == "__main__":
    try:
        sentiment_data = pd.read_csv('./data/sentiment_train_2017_2021.csv')
        return_data = pd.read_csv('./data/return_train_2017_2021.csv')
        print("Data loaded successfully.")
    except FileNotFoundError:
        print("Data files not found. Please check file paths.")
        sentiment_data = None
        return_data = None

    if sentiment_data is not None and return_data is not None:
        model_info = train_model(sentiment_data, return_data)
        
        # For testing prediction, filter the sentiment data to a sample day
        sample_day = pd.to_datetime('2021-06-01')
        preprocessed_sentiment = preprocess_sentiment_data(sentiment_data)
        sentiment_data_today = preprocessed_sentiment[preprocessed_sentiment['Date'] == sample_day].copy()
        
        stock_universe_today = return_data[return_data['Date'] == sample_day]['Ticker'].unique().tolist()
        
        predictions = predict_returns(model_info, sentiment_data_today, stock_universe_today)
        print("Sample predictions:")
        print(predictions.head())

Data loaded successfully.
Preprocessing sentiment data...
Creating features from sentiment data...


C:\Users\Lukea\AppData\Local\Temp\ipykernel_44800\1749845377.py:189: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  special_aggs = df.groupby(['Ticker', 'Date']).apply(


Merging sentiment features with stock returns...
Training fold 214 days -> 209 days...


C:\Users\Lukea\AppData\Local\Temp\ipykernel_44800\1749845377.py:367: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Scalar.cpp:23.)
  print(f"Epoch {epoch:3d} | Train Loss: {loss.item():.6f} | Val Loss: {val_loss.item():.6f}")


Epoch   0 | Train Loss: 0.052968 | Val Loss: 0.018206
Epoch  20 | Train Loss: 0.005217 | Val Loss: 0.002273
Epoch  40 | Train Loss: 0.002580 | Val Loss: 0.001560
Epoch  60 | Train Loss: 0.001950 | Val Loss: 0.001304
Epoch  80 | Train Loss: 0.001621 | Val Loss: 0.001347
Epoch 100 | Train Loss: 0.001491 | Val Loss: 0.001277
Epoch 120 | Train Loss: 0.001368 | Val Loss: 0.001237
Epoch 140 | Train Loss: 0.001287 | Val Loss: 0.001206
Epoch 160 | Train Loss: 0.001228 | Val Loss: 0.001165
Epoch 180 | Train Loss: 0.001136 | Val Loss: 0.001134
Epoch 200 | Train Loss: 0.001079 | Val Loss: 0.001106
Epoch 220 | Train Loss: 0.001035 | Val Loss: 0.001074
Epoch 240 | Train Loss: 0.000997 | Val Loss: 0.001052
Epoch 260 | Train Loss: 0.000964 | Val Loss: 0.001035
Epoch 280 | Train Loss: 0.000926 | Val Loss: 0.001009
Fold final validation loss: 0.000998
Training fold 423 days -> 209 days...
Epoch   0 | Train Loss: 0.015436 | Val Loss: 0.004847
Epoch  20 | Train Loss: 0.002240 | Val Loss: 0.001263
Epoch  

C:\Users\Lukea\AppData\Local\Temp\ipykernel_44800\1749845377.py:189: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  special_aggs = df.groupby(['Ticker', 'Date']).apply(


KeyError: 'Ticker'